In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
    #('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
    #('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    #('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4446


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(9, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0050
Function value obtained: 0.4726
Current minimum: 0.4726
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0021
Function value obtained: 0.4734
Current minimum: 0.4726
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0020
Function value obtained: 0.4750
Current minimum: 0.4726
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4729
Current minimum: 0.4726
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.3651
Function value obtained: 0.4733
Current minimum: 0.4712
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4082
Function value obtained: 0.4717
Current minimum: 0.4712
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.3766
Function value obtained: 0.4737
Current minimum: 0.4712
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4246
Function value obtained: 0.4731
Current minimum: 0.4712
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4189
Function value obtained: 0.4738
Current minimum: 0.4712
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5852
Function value obtained: 0.4735
Current minimum: 0.4711
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.6194
Function value obtained: 0.4736
Current minimum: 0.4711
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.6420
Function value obtained: 0.4740
Current minimum: 0.4711
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.6289
Function value obtained: 0.4732
Current minimum: 0.4711
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.6680
Function value obtained: 0.4711
Current minimum: 0.4711
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2853
Function value obtained: 0.4379
Current minimum: 0.4369
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2717
Function value obtained: 0.4385
Current minimum: 0.4369
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.4008
Function value obtained: 0.4371
Current minimum: 0.4369
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3573
Function value obtained: 0.4364
Current minimum: 0.4364
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.2950
Function value obtained: 0.4365
Current minimum: 0.4364
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.5197
Function value obtained: 0.4391
Current minimum: 0.4364
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.5281
Function value obtained: 0.4373
Current minimum: 0.4364
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.5202
Function value obtained: 0.4374
Current minimum: 0.4364
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5320
Function value obtained: 0.4368
Current minimum: 0.4364
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.5008
Function value obtained: 0.4372
Current minimum: 0.4364
Iteration No: 66 started. Sea

...

Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 0.3312
Function value obtained: 0.4628
Current minimum: 0.4627
Iteration No: 29 started. Searching for the next optimal point.
Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 0.3064
Function value obtained: 0.4660
Current minimum: 0.4627
Iteration No: 30 started. Searching for the next optimal point.
Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3047
Function value obtained: 0.4657
Current minimum: 0.4627
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3606
Function value obtained: 0.4665
Current minimum: 0.4627
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.3205
Function value obtained: 0.4627
Current minimum: 0.4627
Iteration No: 33 started. Sea

Iteration No: 68 ended. Search finished for the next optimal point.
Time taken: 0.4844
Function value obtained: 0.4665
Current minimum: 0.4624
Iteration No: 69 started. Searching for the next optimal point.
Iteration No: 69 ended. Search finished for the next optimal point.
Time taken: 0.4756
Function value obtained: 0.4667
Current minimum: 0.4624
Iteration No: 70 started. Searching for the next optimal point.
Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.4908
Function value obtained: 0.4650
Current minimum: 0.4624
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5114
Function value obtained: 0.4624
Current minimum: 0.4624
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5041
Function value obtained: 0.4655
Current minimum: 0.4624
Iteration No: 73 started. Sea

Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.2237
Function value obtained: 0.4381
Current minimum: 0.4379
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2167
Function value obtained: 0.4380
Current minimum: 0.4379
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2488
Function value obtained: 0.4377
Current minimum: 0.4377
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3090
Function value obtained: 0.4385
Current minimum: 0.4377
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 0.2112
Function value obtained: 0.4396
Current minimum: 0.4377
Iteration No: 16 started. Sea

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.5461
Function value obtained: 0.4382
Current minimum: 0.4372
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4985
Function value obtained: 0.4386
Current minimum: 0.4372
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.4899
Function value obtained: 0.4375
Current minimum: 0.4372
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.4652
Function value obtained: 0.4373
Current minimum: 0.4372
Iteration No: 55 started. Searching for the next optimal point.
Iteration No: 55 ended. Search finished for the next optimal point.
Time taken: 0.4545
Function value obtained: 0.4377
Current minimum: 0.4372
Iteration No: 56 started. Sea

Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7656
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7729
Function value obtained: 0.4384
Current minimum: 0.4371
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.7433
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7476
Function value obtained: 0.4379
Current minimum: 0.4371
Iteration No: 95 started. Searching for the next optimal point.
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.7811
Function value obtained: 0.4382
Current minimum: 0.4371
Iteration No: 96 started. Sea

In [10]:
best_weights

{'cohesion': array([0.1011165 , 0.26595517, 0.04814536, 0.02659552, 0.16078711,
        0.02659552, 0.24217268, 0.06807335, 0.06055879]),
 'syntax': array([0.10424273, 0.22960062, 0.02296006, 0.02296006, 0.17918297,
        0.02296006, 0.22960062, 0.0468925 , 0.14160038]),
 'vocabulary': array([0.19195963, 0.21662669, 0.026775  , 0.026775  , 0.26774996,
        0.09591781, 0.10054803, 0.026775  , 0.0468729 ]),
 'phraseology': array([0.15381284, 0.22667175, 0.02266718, 0.02266718, 0.22667175,
        0.02266718, 0.19127389, 0.02266718, 0.11090107]),
 'grammar': array([0.13565445, 0.30689762, 0.03068976, 0.03068976, 0.30689762,
        0.03068976, 0.03068976, 0.0971015 , 0.03068976]),
 'conventions': array([0.08526474, 0.2671874 , 0.02671874, 0.02671874, 0.18859329,
        0.08819013, 0.15998215, 0.06887621, 0.08846859])}

In [11]:
scores

{'cohesion': 0.4710364857006775,
 'syntax': 0.4362754087307191,
 'vocabulary': 0.4044093162836163,
 'phraseology': 0.4460954740940721,
 'grammar': 0.4624453240801769,
 'conventions': 0.4370945857964947}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4429


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.944736,0.912212,0.908694,0.825234,0.887682
syntax,0.944736,1.000000,0.937108,0.958668,0.916295,0.927717
vocabulary,0.912212,0.937108,1.000000,0.957123,0.876095,0.890875
phraseology,0.908694,0.958668,0.957123,1.000000,0.943555,0.879668
grammar,0.825234,0.916295,0.876095,0.943555,1.000000,0.854531
conventions,0.887682,0.927717,0.890875,0.879668,0.854531,1.000000
